### JsonOutputParser
* 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 parser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [3]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

# 데이터 구조 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드 (2개 이상)")

# 파서 설정
output_parser = JsonOutputParser(pydantic_object=Topic)

In [4]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "title": "Description", "type": "string"}, "hashtags": {"description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc (2\uac1c \uc774\uc0c1)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [5]:
from langchain_core.prompts import ChatPromptTemplate

question = "점심먹고 졸린 이유를 알려주세요."

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format:\n{format_instructions}\nQuestion:\n{question}")
    ]
)

prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

In [6]:
chain = prompt | model | output_parser

answer = chain.invoke({"question":question})

In [8]:
type(answer)

dict

In [7]:
print(answer)

{'description': '점심 식사 후 졸음이 오는 이유는 주로 소화 과정과 관련이 있습니다. 식사 후에는 소화를 위해 혈액이 소화기관으로 집중되면서 뇌로 가는 혈류가 줄어들 수 있습니다. 또한, 식사 후 인슐린 분비가 증가하면서 세로토닌과 멜라토닌 같은 호르몬의 분비가 촉진되어 졸음을 유발할 수 있습니다.', 'hashtags': '#점심졸음 #소화과정'}


### Pydantic을 사용하지 않고 JsonOutputParser 사용

In [10]:
output_parser = JsonOutputParser()

question = "잠에서 깨는 방법에 대해 알려주세요. 방법에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 대답하세요."),
        ("user", "Format:\n{format_instructions}\nQuestion:\n{question}")
    ]
)

# 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt | model | output_parser

response = chain.invoke({"question":question})

print(response)

{'description': '잠에서 깨는 방법에는 여러 가지가 있습니다. 첫째, 알람 시계를 사용하여 규칙적인 시간에 일어나는 습관을 들이세요. 둘째, 자연광을 활용하여 아침 햇살을 받으면 몸이 자연스럽게 깨어납니다. 셋째, 스트레칭이나 가벼운 운동을 통해 몸을 활성화하세요. 넷째, 물을 마셔 수분을 보충하고 몸을 깨우세요. 마지막으로, 상쾌한 음악을 듣거나 향기를 맡아 기분 좋게 하루를 시작하세요.', 'hashtags': ['#알람', '#자연광', '#스트레칭', '#수분보충', '#음악', '#아로마테라피']}
